In [67]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [136]:
import numpy as np
import polars as pl
from sweep.experiment import Experiment, ExperimentType
from model.data import DataModel, KFeaturesDefinition, k_features_factory
from erm.problems.problems import ProblemType
import subprocess
from experiments.data_loading import read_polars_dataframe

Define a datamodel

In [69]:
d = 1000

In [70]:
x_diagonal = KFeaturesDefinition(diagonal=[(0.5,d)])
θ_diagonal = KFeaturesDefinition(diagonal=[(2,d)])
ω_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
δ_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
ν_diagonal = KFeaturesDefinition(diagonal=[(1,d)])

low_low_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

low_robustness_low_usefulness = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=low_low_kwargs,
    name="low_robustness_low_usefulness"
)


In [71]:
x_diagonal = KFeaturesDefinition(diagonal=[(0.5,d)])
θ_diagonal = KFeaturesDefinition(diagonal=[(8,d)])
ω_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
δ_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
ν_diagonal = KFeaturesDefinition(diagonal=[(1,d)])

low_high_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

low_robustness_high_usefulness = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=low_high_kwargs,
    name="low_robustness_high_usefulness"
)


In [72]:
x_diagonal = KFeaturesDefinition(diagonal=[(2,d)])
θ_diagonal = KFeaturesDefinition(diagonal=[(0.5,d)])
ω_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
δ_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
ν_diagonal = KFeaturesDefinition(diagonal=[(1,d)])

high_low_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

high_robustness_low_usefulness = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=high_low_kwargs,
    name="high_robustness_low_usefulness"
)


In [73]:
x_diagonal = KFeaturesDefinition(diagonal=[(2,d)])
θ_diagonal = KFeaturesDefinition(diagonal=[(2,d)])
ω_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
δ_diagonal = KFeaturesDefinition(diagonal=[(1,d)])
ν_diagonal = KFeaturesDefinition(diagonal=[(1,d)])

high_high_kwargs = {
    "x_diagonal": x_diagonal,
    "θ_diagonal": θ_diagonal,
    "ω_diagonal": ω_diagonal,
    "δ_diagonal": δ_diagonal,
    "ν_diagonal": ν_diagonal,
}

high_robustness_high_usefulness = DataModel(
    d,
    normalize_matrices=False,
    data_model_factory=k_features_factory,
    factory_kwargs=high_high_kwargs,
    name="high_robustness_high_usefulness"
)


Define an experiment

In [74]:
experiment = Experiment(
    state_evolution_repetitions=1,
    erm_repetitions=1,
    alphas=np.logspace(-0.2,2,10),
    epsilons=np.array([0.0, 0.1, 0.2, 0.3]),
    lambdas=np.array([0.001]),
    taus=np.array([0.05]),
    d=d,
    experiment_type=ExperimentType.Sweep,
    data_models = [low_robustness_low_usefulness, low_robustness_high_usefulness, high_robustness_low_usefulness, high_robustness_high_usefulness],
    test_against_epsilons=np.array([0.2]),
    erm_problem_type=ProblemType.Logistic,
    gamma_fair_error=0.01,
    name="feature_combinations"
)
experiment_json = experiment.to_json()

In [75]:
with open("feature_combinations.json", "w") as f:
    f.write(experiment_json)

In [76]:
# venv_python = ".venv/bin/python"
# command = ["mpiexec", "-n", "5", venv_python, "sweep/run_sweep.py", "--json", experiment_json, "--log-level", "INFO"]
# subprocess.run(command)

In [196]:
df = read_polars_dataframe(experiment.name)

/Users/kasimirtanner/Documents/LocalCode/Code_PDM/AISTATS/experiments/data_loading.py:40: UserWarning: `to_struct()` should be passed a list of field names to avoid query errors in subsequent operations (e.g. <struct operation> not supported for dtype Unknown)
  df = df.rename({"epsilon_g_0":"epsilon_g"})


In [197]:
df.shape

(160, 148)

In [198]:
df

alpha,epsilon,tau,lam,epsilon_g,data_model_name,id,date,task_type,erm_problem_type,test_against_epsilons,d,values,gamma_fair_error,gamma,generalization_error,adversarial_generalization_errors,training_error,training_loss,test_losses,m,q,sigma,A,P,F,m_hat,q_hat,sigma_hat,A_hat,F_hat,P_hat,n_m,n_q,n_sigma,n_A,n_P,…,adversarial_generalization_errors_teacher,adversarial_generalization_errors_overlap,fair_adversarial_errors,training_error_erm,boundary_loss_train,boundary_loss_test_es,training_loss_erm,test_losses_erm,duration_erm,id_std_erm,date_std_erm,task_type_std_erm,erm_problem_type_std_erm,test_against_epsilons_std_erm,d_std_erm,values_std_erm,gamma_fair_error_std_erm,gamma_std_erm,ρ_std,m_std_erm,F_std_erm,Q_std,A_std_erm,P_std_erm,angle_std_erm,generalization_error_erm_std,generalization_error_overlap_std,adversarial_generalization_errors_std_erm,adversarial_generalization_errors_teacher_std,adversarial_generalization_errors_overlap_std,fair_adversarial_errors_std,training_error_std_erm,boundary_loss_train_std,boundary_loss_test_es_std,training_loss_std_erm,test_losses_std_erm,duration_std_erm
f64,f64,f64,f64,f64,str,str,str,str,str,list[f64],f64,struct[0],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,…,f64,f64,f64,f64,f64,f64,f64,f64,f64,str,str,str,str,list[f64],f64,struct[0],f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64
0.630957,0.0,0.05,0.001,0.2,"""low_robustness_high_usefulness""",null,null,null,null,null,1000.0,{},0.01,1.0,0.30329,0.423484,0.0,0.011026,4.362861,4.427787,14.59142,291.863443,29.18284,29.18284,8.855575,0.003793,0.000114,0.001426,0.0,0.0,0.0,4.427787,14.59142,291.863443,29.18284,29.18284,…,0.1295,0.595167,0.4159,0.0,0.0,18.46232,0.010394,1.173841,18.655912,null,null,null,null,null,null,{},null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
100.0,0.0,0.05,0.001,0.2,"""low_robustness_high_usefulness""",null,null,null,null,null,1000.0,{},0.01,1.0,0.010747,0.222703,0.005813,0.022581,351.075538,61.262941,938.770702,0.917681,1877.541404,1877.541404,122.525881,16.689601,0.574405,1.087703,0.0,0.0,0.0,61.262941,938.770702,0.917681,1877.541404,1877.541404,…,0.23,0.570608,0.1025,0.00573,0.0,2094.717195,0.022662,1.051357,59.017711,null,null,null,null,null,null,{},null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.994843,0.0,0.05,0.001,0.2,"""high_robustness_low_usefulness""",null,null,null,null,null,1000.0,{},0.01,1.0,0.076062,0.144942,1.1302e-29,0.008788,18.103042,17.194694,312.422403,75.169449,156.211201,156.211201,8.597347,0.228746,0.002967,0.012803,0.0,0.0,0.0,17.194694,312.422403,75.169449,156.211201,156.211201,…,0.1117,-0.431544,0.1298,0.0,0.0,340.524849,0.008571,0.525551,49.044804,null,null,null,null,null,null,{},null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
32.442261,0.2,0.05,0.001,0.2,"""high_robustness_low_usefulness""",null,null,null,null,null,1000.0,{},0.01,1.0,0.028747,0.114271,0.014585,4.314174,4.38137,10.504156,110.964747,0.565413,55.482374,55.482374,5.252078,18.577849,1.962684,1.71634,0.0,0.0,0.103558,10.504156,110.964747,0.565413,55.482374,55.482374,…,0.1077,-0.441666,0.0661,0.014703,406.354458,125.488195,0.184422,0.203569,30.646932,null,null,null,null,null,null,{},null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
5.994843,0.3,0.05,0.001,0.2,"""high_robustness_high_usefulnes…",null,null,null,null,null,1000.0,{},0.01,1.0,0.073855,0.143106,0.004528,11.532701,6.177271,22.948687,138.924017,4.642131,69.462009,69.462009,11.474343,1.235892,0.337061,0.202323,0.0,0.0,0.02519,22.948687,138.924017,4.642131,69.462009,69.462009,…,0.1135,0.545327,0.1257,0.00317,154.948698,159.249107,0.179152,0.392646,35.183068,null,null,null,null,null,null,{},null,null,null,null,null,null,null,null,null,null,null,null,null,n